# Módulo 3: Visualização de Dados com Matplotlib e Seaborn 📊

Bem-vindo(a) ao Módulo 3! Depois de calcularmos métricas financeiras importantes, chegou a hora de **visualizar** esses resultados. Um gráfico bem-feito comunica uma ideia muito mais rápido do que uma tabela de números.

Hoje, usaremos as bibliotecas `Matplotlib` e `Seaborn` para criar gráficos profissionais e extrair insights ainda mais profundos dos dados da "Sabor Artesanal Lanchonete".

### Passo 1: Preparando o Ambiente

Como sempre, começamos importando as bibliotecas e carregando todos os nossos dados. Vamos também refazer alguns cálculos do Módulo 2 para termos todas as informações necessárias para os gráficos.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configuração para melhorar a aparência dos gráficos do Seaborn
sns.set_theme(style="whitegrid")

# Carregando os dados
base_url = 'https://raw.githubusercontent.com/Miriam1s/mini_curso_inobar/main/dados/'
df_vendas = pd.read_csv(base_url + 'vendas.csv')
df_produtos = pd.read_csv(base_url + 'produtos.csv')
df_funcionarios = pd.read_csv(base_url + 'funcionarios.csv')
df_despesas = pd.read_csv(base_url + 'despesas_operacionais.csv')
df_avaliacoes = pd.read_csv(base_url + 'avaliacoes.csv')

# --- Recriando os cálculos do Módulo 2 ---

# 1. DataFrame de vendas completo com Receita, Custo e Lucro
df_vendas_completo = pd.merge(df_vendas, df_produtos, on='ID_Produto')
df_vendas_completo['Receita'] = df_vendas_completo['Preco_Venda_Unitario'] * df_vendas_completo['Quantidade']
df_vendas_completo['Custo'] = df_vendas_completo['Custo_Producao_Unitario'] * df_vendas_completo['Quantidade']
df_vendas_completo['Lucro'] = df_vendas_completo['Receita'] - df_vendas_completo['Custo']

print("Ambiente e dados preparados com sucesso!")

### Gráfico 1: Análise de Performance dos Produtos

Vamos criar um gráfico de barras para comparar a **Receita** e o **Lucro** total de cada produto.

In [ ]:
# Agrupando os dados por produto
analise_produtos = df_vendas_completo.groupby('Nome_Produto').agg(
    Receita_Total=('Receita', 'sum'),
    Lucro_Total=('Lucro', 'sum')
).sort_values(by='Receita_Total', ascending=False)

# Reformatando os dados para o gráfico
analise_produtos_melted = analise_produtos.reset_index().melt(id_vars='Nome_Produto', var_name='Metrica', value_name='Valor')

# Criando o gráfico de barras agrupado
plt.figure(figsize=(14, 8))
sns.barplot(data=analise_produtos_melted, x='Valor', y='Nome_Produto', hue='Metrica', palette='viridis')

# Adicionando títulos e rótulos
plt.title('Receita vs. Lucro por Produto em Outubro', fontsize=16)
plt.xlabel('Valor Total (R$)', fontsize=12)
plt.ylabel('Produto', fontsize=12)
plt.legend(title='Métrica')

# Exibindo o gráfico
plt.show()

### Gráfico 2: Faturamento por Dia da Semana

Um gráfico de linhas é ideal para mostrar a performance de vendas ao longo da semana.

In [ ]:
# Preparando os dados de dia da semana
df_vendas_completo['Data'] = pd.to_datetime(df_vendas_completo['Data'])
df_vendas_completo['Dia_Semana'] = df_vendas_completo['Data'].dt.day_name(locale='pt_BR.utf8')
dias_ordem = ['segunda-feira', 'terça-feira', 'quarta-feira', 'quinta-feira', 'sexta-feira', 'sábado', 'domingo']
receita_por_dia = df_vendas_completo.groupby('Dia_Semana')['Receita'].sum().reindex(dias_ordem).fillna(0)

# Criando o gráfico de linha
plt.figure(figsize=(12, 6))
sns.lineplot(x=receita_por_dia.index, y=receita_por_dia.values, marker='o', color='royalblue', linewidth=2.5)

# Adicionando títulos e rótulos
plt.title('Faturamento por Dia da Semana', fontsize=16)
plt.xlabel('Dia da Semana', fontsize=12)
plt.ylabel('Receita Total (R$)', fontsize=12)
plt.xticks(rotation=45)

plt.show()

### Gráfico 3: Distribuição das Formas de Pagamento

Um gráfico de pizza (ou de rosca) é perfeito para mostrar a proporção de cada forma de pagamento no total de transações.

In [ ]:
# Contando a frequência de cada forma de pagamento
frequencia_pagamento = df_vendas_completo['Forma_Pagamento'].value_counts()

# Criando o gráfico de pizza
plt.figure(figsize=(10, 8))
plt.pie(frequencia_pagamento, labels=frequencia_pagamento.index, autopct='%1.1f%%', startangle=140, colors=sns.color_palette('pastel'))

# Adicionando um círculo no centro para criar um gráfico de rosca (donut chart)
my_circle=plt.Circle( (0,0), 0.7, color='white')
p=plt.gcf()
p.gca().add_artist(my_circle)

# Adicionando título
plt.title('Distribuição das Formas de Pagamento', fontsize=16)
plt.ylabel('') # Remove o rótulo do eixo y

plt.show()

### Gráfico 4: Breakdown das Despesas Operacionais

Vamos visualizar para onde o dinheiro está indo, separando salários das outras despesas.

In [ ]:
# Calculando os totais
total_salarios = df_funcionarios['Salario_Bruto'].sum()
total_outras_despesas = df_despesas['Valor'].sum()

# Criando um DataFrame para o gráfico
despesas_breakdown = pd.DataFrame({
    'Tipo de Despesa': ['Salários', 'Outras Despesas'],
    'Valor': [total_salarios, total_outras_despesas]
})

# Criando o gráfico de barras
plt.figure(figsize=(8, 6))
sns.barplot(data=despesas_breakdown, x='Tipo de Despesa', y='Valor', palette='Reds_r')

# Adicionando títulos e rótulos
plt.title('Breakdown das Despesas Operacionais', fontsize=16)
plt.xlabel('Tipo de Despesa', fontsize=12)
plt.ylabel('Valor Total (R$)', fontsize=12)

plt.show()

### Gráfico 5: Resumo Financeiro - O Caminho até o Lucro Líquido

Este gráfico resume toda a nossa análise, mostrando como a receita se transforma em lucro líquido após a dedução de todos os custos e despesas.

In [ ]:
# Calculando os valores finais
receita_bruta_total = df_vendas_completo['Receita'].sum()
custo_total_produtos = df_vendas_completo['Custo'].sum()
lucro_bruto = receita_bruta_total - custo_total_produtos
despesas_totais = total_salarios + total_outras_despesas
lucro_liquido = lucro_bruto - despesas_totais

# Criando o DataFrame para o gráfico
resumo_financeiro = pd.DataFrame({
    'Métrica': ['Receita Bruta', 'Lucro Bruto', 'Lucro Líquido'],
    'Valor': [receita_bruta_total, lucro_bruto, lucro_liquido]
})

# Criando o gráfico de barras
plt.figure(figsize=(10, 6))
ax = sns.barplot(data=resumo_financeiro, x='Métrica', y='Valor', palette='Greens_r')

# Adicionando os valores em cima das barras
for p in ax.patches:
    ax.annotate(f'R$ {p.get_height():,.2f}', 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points')

# Adicionando títulos e rótulos
plt.title('Resumo Financeiro do Mês', fontsize=16)
plt.xlabel('Métrica Financeira', fontsize=12)
plt.ylabel('Valor (R$)', fontsize=12)

plt.show()

### Gráfico 6: Satisfação dos Clientes por Produto

Vamos visualizar a nota média de cada produto para entender a percepção de qualidade dos clientes.

In [ ]:
# Juntando avaliações e produtos
df_avaliacoes_completo = pd.merge(df_avaliacoes, df_produtos, on='ID_Produto')
media_avaliacoes = df_avaliacoes_completo.groupby('Nome_Produto')['Nota'].mean().sort_values(ascending=True)

# Criando o gráfico de barras horizontais
plt.figure(figsize=(12, 7))
ax = sns.barplot(x=media_avaliacoes.values, y=media_avaliacoes.index, palette='coolwarm_r')

# Adicionando os valores no final das barras
ax.bar_label(ax.containers[0], fmt='%.2f')

# Adicionando títulos e rótulos
plt.title('Média de Avaliação por Produto', fontsize=16)
plt.xlabel('Nota Média (de 1 a 5)', fontsize=12)
plt.ylabel('Produto', fontsize=12)
plt.xlim(0, 5.5) # Define o limite do eixo x para ser de 0 a 5.5 para dar espaço para o rótulo

plt.show()